In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# אלגוריתם קירוב אופטימיזציה קוונטי

*הערכת שימוש: 22 דקות על מעבד Heron r3 (הערה: זוהי הערכה בלבד. זמן הריצה שלך עשוי להשתנות.)*
## רקע
מדריך זה מדגים כיצד ליישם את **אלגוריתם הקירוב לאופטימיזציה הקוונטית (QAOA)** – שיטה איטרטיבית היברידית (קוונטית-קלאסית) – בהקשר של תבניות Qiskit. תפתור תחילה את בעיית **החיתוך המקסימלי** (או **Max-Cut**) עבור גרף קטן ולאחר מכן תלמד כיצד להריץ אותה בקנה מידה שימושי. כל ריצות החומרה במדריך אמורות להסתיים בתוך מגבלת הזמן של התוכנית החופשית הזמינה (Open Plan).

בעיית Max-Cut היא בעיית אופטימיזציה שקשה לפתור (ליתר דיוק, זוהי בעיית *NP-hard*) עם מספר יישומים שונים באשכולות, מדעי הרשת ופיזיקה סטטיסטית. מדריך זה בוחן גרף של קודקודים המחוברים על ידי קשתות, ומטרתו לחלק את הקודקודים לשתי קבוצות כך שמספר הקשתות שחוצות את החיתוך הזה יהיה מקסימלי.

![Illustration of a max-cut problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## דרישות
לפני תחילת מדריך זה, ודא שיש לך את הדברים הבאים מותקנים:
- Qiskit SDK גרסה 1.0 ומעלה, עם תמיכה ב-[ויזואליזציה](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime גרסה 0.22 ומעלה (`pip install qiskit-ibm-runtime`)

בנוסף, תזדקק לגישה לאינסטנס ב-[IBM Quantum Platform](/guides/cloud-setup). שים לב שמדריך זה לא ניתן להרצה בתוכנית החופשית (Open Plan), מכיוון שהוא מריץ עומסי עבודה באמצעות [סשנים](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session), הזמינים רק עם גישה לתוכנית Premium.
## הגדרה

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## חלק I. QAOA בקנה מידה קטן
החלק הראשון של המדריך משתמש בבעיית Max-Cut בקנה מידה קטן כדי להמחיש את השלבים לפתרון בעיית אופטימיזציה באמצעות מחשב קוונטי.

כדי לתת הקשר לפני מיפוי הבעיה הזו לאלגוריתם קוונטי, ניתן להבין טוב יותר כיצד בעיית Max-Cut הופכת לבעיית אופטימיזציה קומבינטורית קלאסית על ידי שקול תחילה של מינימיזציה של פונקציה $f(x)$

$$
\min_{x\in {0, 1}^n}f(x),
$$

כאשר הקלט $x$ הוא וקטור שמרכיביו מתאימים לכל קודקוד של גרף. לאחר מכן, הגבל כל אחד מהמרכיבים האלה להיות $0$ או $1$ (המייצגים נכללים או לא נכללים בחיתוך). מקרה הדגמה בקנה מידה קטן זה משתמש בגרף עם $n=5$ קודקודים.

ניתן לכתוב פונקציה של זוג קודקודים $i,j$ המצביעה אם הקשת המתאימה $(i,j)$ נמצאת בחיתוך. לדוגמה, הפונקציה $x_i + x_j - 2 x_i x_j$ היא 1 רק אם אחד מ-$x_i$ או $x_j$ הוא 1 (כלומר שהקשת נמצאת בחיתוך) ואפס אחרת. הבעיה של מקסום הקשתות בחיתוך ניתנת לניסוח כ

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

אשר ניתן לכתוב מחדש כמינימיזציה בצורה

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

המינימום של $f(x)$ במקרה זה הוא כאשר מספר הקשתות שחוצות את החיתוך הוא מקסימלי. כפי שאתה יכול לראות, אין שום דבר הקשור למחשוב קוונטי עדיין. עליך לנסח מחדש את הבעיה הזו למשהו שמחשב קוונטי יכול להבין.
אתחל את הבעיה שלך על ידי יצירת גרף עם $n=5$ קודקודים.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### שלב 1: מיפוי קלטים קלאסיים לבעיה קוונטית
השלב הראשון של התבנית הוא למפות את הבעיה הקלאסית (גרף) למעגלים ואופרטורים קוונטיים. כדי לעשות זאת, ישנם שלושה שלבים עיקריים:

1. השתמש בסדרה של ניסוחים מתמטיים מחדש, כדי לייצג את הבעיה באמצעות סימון בעיות אופטימיזציה בינארית בלתי מוגבלת ריבועית (QUBO).
2. כתוב מחדש את בעיית האופטימיזציה כהמילטוניאן שעבורו מצב היסוד מתאים לפתרון שממזער את פונקציית העלות.
3. צור מעגל קוונטי שיכין את מצב היסוד של ההמילטוניאן הזה בתהליך דומה ל-quantum annealing.

**הערה:** במתודולוגיית QAOA, אתה רוצה בסופו של דבר אופרטור (**המילטוניאן**) המייצג את **פונקציית העלות** של האלגוריתם ההיברידי שלנו, כמו גם מעגל פרמטרי (**Ansatz**) המייצג מצבים קוונטיים עם פתרונות מועמדים לבעיה. ניתן לדגום מהמצבים המועמדים האלה ולאחר מכן להעריך אותם באמצעות פונקציית העלות.

#### גרף &rarr; בעיית אופטימיזציה
השלב הראשון של המיפוי הוא שינוי סימון. הדבר הבא מביע את הבעיה בסימון QUBO:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

כאשר $Q$ היא מטריצה $n\times n$ של מספרים ממשיים, $n$ מתאים למספר הקודקודים בגרף שלך, $x$ הוא הוקטור של משתנים בינאריים שהוצגו לעיל, ו-$x^T$ מציין את התמורה של הוקטור $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### בעיית אופטימיזציה &rarr; המילטוניאן
לאחר מכן ניתן לנסח מחדש את בעיית QUBO כ**המילטוניאן** (כאן, מטריצה המייצגת את האנרגיה של מערכת):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**שלבי ניסוח מחדש מבעיית QAOA להמילטוניאן**
</summary>

כדי להדגים כיצד ניתן לכתוב מחדש את בעיית QAOA בדרך זו, החלף תחילה את המשתנים הבינאריים $x_i$ לקבוצה חדשה של משתנים $z_i\in{-1, 1}$ באמצעות

$$
x_i = \frac{1-z_i}{2}.
$$

כאן ניתן לראות שאם $x_i$ הוא $0$, אז $z_i$ חייב להיות $1$. כאשר ה-$x_i$ מוחלפים ב-$z_i$ בבעיית האופטימיזציה ($x^TQx$), ניתן לקבל ניסוח שווה ערך.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

כעת אם נגדיר $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$, נסיר את הקדם-גורם ואת האיבר הקבוע $n^2$, נגיע לשני הניסוחים השווים של אותה בעיית אופטימיזציה.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

כאן, $b$ תלוי ב-$Q$. שים לב שכדי לקבל את $z^TQz + b^Tz$ הורדנו את הגורם 1/4 והיסט קבוע של $n^2$ שאינם משחקים תפקיד באופטימיזציה.

כעת, כדי לקבל ניסוח קוונטי של הבעיה, הפוך את המשתנים $z_i$ למטריצת Pauli $Z$, כגון מטריצה $2\times 2$ בצורה

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

כאשר תחליף את המטריצות האלה בבעיית האופטימיזציה לעיל, תקבל את ההמילטוניאן הבא

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*זכור גם שמטריצות $Z$ משובצות במרחב החישוב של המחשב הקוונטי, כלומר מרחב הילברט בגודל $2^n\times 2^n$. לכן, עליך להבין מונחים כמו $Z_iZ_j$ כמכפלה טנזורית $Z_i\otimes Z_j$ המשובצת במרחב הילברט $2^n\times 2^n$. לדוגמה, בבעיה עם חמישה משתני החלטה, המונח $Z_1Z_3$ מובן כ-$I\otimes Z_3\otimes I\otimes Z_1\otimes I$ כאשר $I$ היא מטריצת הזהות $2\times 2$.*
</details>

המילטוניאן זה נקרא **המילטוניאן של פונקציית העלות**. יש לו את התכונה שמצב היסוד שלו מתאים לפתרון ש**ממזער את פונקציית העלות $f(x)$**.
לכן, כדי לפתור את בעיית האופטימיזציה שלך, עליך כעת להכין את מצב היסוד של $H_C$ (או מצב עם חפיפה גבוהה איתו) במחשב הקוונטי. לאחר מכן, דגימה ממצב זה תניב, בהסתברות גבוהה, את הפתרון ל-$min~f(x)$.
כעת בוא נשקול את ההמילטוניאן $H_C$ עבור בעיית **Max-Cut**. תן לכל קודקוד של הגרף להיות משויך ל-qubit במצב $|0\rangle$ או $|1\rangle$, כאשר הערך מציין את הקבוצה בה הקודקוד נמצא. המטרה של הבעיה היא למקסם את מספר הקשתות $(v_1, v_2)$ שעבורן $v_1 = |0\rangle$ ו-$v_2 = |1\rangle$, או להיפך. אם נשייך את אופרטור $Z$ לכל qubit, כאשר

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

אזי קשת $(v_1, v_2)$ שייכת לחיתוך אם הערך העצמי של $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$; במילים אחרות, ה-qubits המשויכים ל-$v_1$ ו-$v_2$ שונים. באופן דומה, $(v_1, v_2)$ לא שייכת לחיתוך אם הערך העצמי של $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$. שים לב שאנחנו לא דואגים למצב ה-qubit המדויק המשויך לכל קודקוד, אלא דואגים רק אם הם זהים או לא לאורך קשת. בעיית Max-Cut דורשת מאיתנו למצוא השמה של ה-qubits על הקודקודים כך שהערך העצמי של ההמילטוניאן הבא הוא ממוזער
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

במילים אחרות, $b_i = 0$ עבור כל $i$ בבעיית Max-Cut. הערך של $Q_{ij}$ מציין את משקל הקשת. במדריך זה אנו שוקלים גרף לא משוקלל, כלומר $Q_{ij} = 1.0$ עבור כל $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### שלב 2: אופטימיזציה של הבעיה לביצוע על חומרה קוונטית
המעגל לעיל מכיל סדרה של הפשטות שימושיות לחשיבה על אלגוריתמים קוונטיים, אך לא ניתן להפעלה על החומרה. כדי שניתן יהיה להריץ על QPU, המעגל צריך לעבור סדרה של פעולות המרכיבות את שלב **הטרנספילציה** או **אופטימיזציית המעגל** של התבנית.

ספריית Qiskit מציעה סדרה של **מעברי טרנספילציה** הנותנים מענה למגוון רחב של טרנספורמציות מעגל. עליך לוודא שהמעגל שלך **מאופטם** למטרה שלך.

הטרנספילציה עשויה לכלול מספר שלבים, כגון:

* **מיפוי התחלתי** של ה-qubits במעגל (כגון משתני החלטה) ל-qubits פיזיים על ההתקן.
* **פריסה** של ההוראות במעגל הקוונטי להוראות מקוריות לחומרה שה-backend מבין.
* **ניתוב** של כל qubits במעגל שמתקשרים ל-qubits פיזיים שסמוכים זה לזה.
* **דיכוי שגיאות** על ידי הוספת שערים של qubit יחיד כדי לדכא רעש עם decoupling דינמי.

מידע נוסף על טרנספילציה זמין ב[תיעוד](/guides/transpile) שלנו.

הקוד הבא ממיר ומייעל את המעגל המופשט לפורמט המוכן להרצה על אחד מההתקנים הזמינים דרך הענן באמצעות **שירות Qiskit IBM Runtime**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### שלב 3: ביצוע באמצעות פרימיטיבים של Qiskit
בזרימת העבודה של QAOA, פרמטרי QAOA האופטימליים נמצאים בלולאת אופטימיזציה איטרטיבית, שמריצה סדרה של הערכות מעגל ומשתמשת באופטימייזר קלאסי כדי למצוא את פרמטרי $\beta_k$ ו-$\gamma_k$ האופטימליים. לולאת הביצוע הזו מופעלת באמצעות השלבים הבאים:

1. הגדר את הפרמטרים ההתחלתיים
2. צור אינסטנס `Session` חדש המכיל את לולאת האופטימיזציה ואת הפרימיטיב המשמש לדגימת המעגל
3. לאחר שנמצאה קבוצה אופטימלית של פרמטרים, הפעל את המעגל פעם אחרונה כדי לקבל התפלגות סופית שתשמש בשלב העיבוד-לאחר.
#### הגדרת מעגל עם פרמטרים התחלתיים
נתחיל עם פרמטרים שנבחרו באופן שרירותי.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### הגדרת backend ופרימיטיב ביצוע
השתמש ב**פרימיטיבים של Qiskit Runtime** כדי לתקשר עם backends של IBM&reg;. שני הפרימיטיבים הם Sampler ו-Estimator, ובחירת הפרימיטיב תלויה באיזה סוג של מדידה אתה רוצה להריץ על המחשב הקוונטי. עבור מינימיזציה של $H_C$, השתמש ב-Estimator מכיוון שהמדידה של פונקציית העלות היא פשוט ערך התוחלת של $\langle H_C \rangle$.
#### הפעלה
הפרימיטיבים מציעים מגוון [מצבי ביצוע](/guides/execution-modes) לתזמון עומסי עבודה על התקנים קוונטיים, וזרימת עבודה של QAOA רצה באופן איטרטיבי בסשן.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

ניתן לחבר את פונקציית העלות המבוססת על sampler לשגרת המינימיזציה של SciPy כדי למצוא את הפרמטרים האופטימליים.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

לאחר שמצאת את הפרמטרים האופטימליים עבור המעגל, ניתן להקצות את הפרמטרים האלה ולדגום את ההתפלגות הסופית המתקבלת עם הפרמטרים המאופטמים. כאן צריך להשתמש בפרימיטיב *Sampler* מכיוון שזוהי התפלגות ההסתברות של מדידות bitstring המתאימות לחיתוך האופטימלי של הגרף.

**הערה:** זה אומר הכנת מצב קוונטי $\psi$ במחשב ולאחר מכן מדידתו. מדידה תקרוס את המצב למצב בסיס חישובי יחיד - לדוגמה, `010101110000...` - המתאים לפתרון מועמד $x$ לבעיית האופטימיזציה ההתחלתית שלנו ($\max f(x)$ או $\min f(x)$ בהתאם למשימה).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### שלב 4: עיבוד-לאחר והחזרת תוצאה בפורמט קלאסי רצוי
שלב העיבוד-לאחר מפרש את פלט הדגימה כדי להחזיר פתרון לבעיה המקורית שלך. במקרה זה, אתה מעוניין ב-bitstring בעל ההסתברות הגבוהה ביותר שכן זה קובע את החיתוך האופטימלי. הסימטריות בבעיה מאפשרות ארבעה פתרונות אפשריים, ותהליך הדגימה יחזיר אחד מהם בהסתברות מעט גבוהה יותר, אך ניתן לראות בהתפלגות המצויירת למטה שארבעה מה-bitstrings הם ללא ספק סבירים יותר מהשאר.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### ויזואליזציה של החיתוך הטוב ביותר
מה-bitstring האופטימלי, ניתן אז לדמיין את החיתוך הזה על הגרף המקורי.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

וחשב את ערך החיתוך:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## חלק II. הגדלת קנה המידה!
יש לך גישה להתקנים רבים עם למעלה מ-100 qubits ב-IBM Quantum&reg; Platform. בחר אחד עליו לפתור את Max-Cut על גרף משוקלל של 100 קודקודים. זוהי בעיה "בקנה מידה שימושי". השלבים לבניית זרימת העבודה עוקבים אחר אותם שלבים כמו למעלה, אך עם גרף גדול בהרבה.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### שלב 1: מיפוי קלטים קלאסיים לבעיה קוונטית
#### גרף &rarr; המילטוניאן
ראשית, המר את הגרף שברצונך לפתור ישירות להמילטוניאן המתאים ל-QAOA.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### המילטוניאן &rarr; מעגל קוונטי

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### שלב 2: אופטימיזציה של הבעיה לביצוע קוונטי
כדי להגדיל את קנה המידה של שלב אופטימיזציית המעגל לבעיות בקנה מידה שימושי, ניתן לנצל את אסטרטגיות הטרנספילציה בעלות הביצועים הגבוהים שהוצגו ב-Qiskit SDK גרסה 1.0. כלים אחרים כוללים את שירות הטרנספילציה החדש עם [מעברי טרנספילציה משופרי AI](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

לאחר שנמצאו הפרמטרים האופטימליים מהרצת QAOA על ההתקן, הקצה את הפרמטרים למעגל.